In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir:Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [2]:
%pwd

'/home/tuananh/MLOPs-for-Deep-Learning/research'

In [3]:
import os

os.chdir("../")

In [4]:
from src.constants import *
from src.utils.main_utils import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)-> DataIngestionConfig:
        
        create_directories([self.config.data_ingestion.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=self.config.data_ingestion.root_dir,
            source_url=self.config.data_ingestion.source_URL,
            local_data_file=self.config.data_ingestion.local_data_file,
            unzip_dir=self.config.data_ingestion.unzip_dir
        )
        return data_ingestion_config

In [5]:
import os
import zipfile
import gdown
from src.logger import logging
from src.utils.main_utils import get_size

class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_data(self)->str:
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logging.info(f"Downloading data from {dataset_url} to {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, output=zip_download_dir)

            logging.info(f"Downloading data from {dataset_url} to {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file,"r") as f:
            f.extractall(unzip_path)
        

In [6]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-11-17 16:46:51,797: INFO: main_utils: yaml file: config/config.yaml loaded successfully]
[2024-11-17 16:46:51,800: INFO: main_utils: yaml file: config/params.yaml loaded successfully]
[2024-11-17 16:46:51,801: INFO: main_utils: directory: artifacts created successfully]
[2024-11-17 16:46:51,802: INFO: main_utils: directory: artifacts/data_ingestion created successfully]
[2024-11-17 16:46:51,803: INFO: 3642229577: Downloading data from https://drive.google.com/file/d/1s8i4aYQZn5ySemj9yyysamgzlMz6hupJ/view?usp=sharing to artifacts/data_ingestion/data.zip]


Downloading...
From: https://drive.google.com/uc?/export=download&id=1s8i4aYQZn5ySemj9yyysamgzlMz6hupJ
To: /home/tuananh/MLOPs-for-Deep-Learning/artifacts/data_ingestion/data.zip
100%|██████████| 23.2M/23.2M [00:03<00:00, 6.66MB/s]

[2024-11-17 16:47:16,656: INFO: 3642229577: Downloading data from https://drive.google.com/file/d/1s8i4aYQZn5ySemj9yyysamgzlMz6hupJ/view?usp=sharing to artifacts/data_ingestion/data.zip]
